In [12]:
import datetime
from datetime import date
from datetime import datetime
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
# import altair as alt
import time
import itertools
import numpy as np
import operator
from datetime import timedelta
from ipywidgets import interact, IntSlider
from IPython.display import display

# pd.options.display.max_columns = 5000
# pd.options.display.max_rows = 5000

In [29]:
# Load Dataset

# Renzo's Path
# dataset = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/complete_final_df_5.csv')

# Sravan's Path
dataset = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/complete_final_df_5.csv')

In [30]:
dataset.shape

(1680385, 68)

In [31]:
# import matplotlib.pyplot as plt
# from matplotlib.backends.backend_pdf import PdfPages

# fig, ax =plt.subplots(figsize=(12,4))
# ax.axis('tight')
# ax.axis('off')
# the_table = ax.table(cellText=dataset.values,colLabels=dataset.columns,loc='center')

# pp = PdfPages("foo.pdf")
# pp.savefig(fig, bbox_inches='tight')
# pp.close()

In [32]:
# Dropping zero lines
dataset = dataset[dataset['FBRefID'] != 0]

# ['name', 'FBRefID', 'date', 'agg_week', 'agg_year', 'Injury',
# 'injury_week', 'injury_year', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh',
# 'SoT', 'CrdY', 'CrdR', 'Touches', 'Press', 'Tkl', 'Int', 'Blocks', 'xG',
# 'npxG', 'xA', 'SCA', 'GCA', 'Cmp', 'Att', 'Prog', 'Carries', 'Prog.1',
# 'Succ', 'Att.1', 'Fls', 'Fld', 'Off', 'Crs', 'TklW', 'OG', 'PKwon',
# 'PKcon', 'Won', 'Loss', 'Draw', 'release_week', 'was_match', 'Height',
# 'Weight', 'Birth', 'cum_week', 'defender', 'attacker', 'midfielder',
# 'goalkeeper', 'right_foot', 'left_foot', 'injury_count', 'cum_injury',
# 'age', 'first_week', 'first_year', 'week_diff', 'year_diff',
# 'unique_injury_count', 'cum_injury_total', 'previous_injury_week',
# 'weeks_since_injury']

In [33]:
# Creating the Column Age from current_date and birth and cleaning unneeded features

# dates = (dataset['agg_year'] * 100 + dataset['agg_week']).astype(int)

# dataset['date'] = pd.to_datetime(dates.astype(str) + '0', format='%Y%W%w')

# dataset['age'] = round((pd.to_datetime(dataset['date']) - pd.to_datetime(dataset['Birth'])) / timedelta(days=365), 0)

# dataset['retired'] = np.where(dataset['Retired since:'].isnull(), 0, 1)

# dataset.drop(columns = ['Retired since:', 'Position:', 'PlayerName', 'FootAbility', 'Ban since:', 'current_year', 'current_week', 'FBRefId', 'date', 'Birth'], inplace = True)


# dataset[dataset['FBRefID'] == '71672fa0'].tail(10)

In [34]:
# Sorting values in dataset_for_model
dataset = dataset.sort_values(['FBRefID', 'agg_year', 'agg_week'])

In [35]:
# Creating 'injured' column

dataset.loc[dataset['Injury'] != '0', 'injured'] = 1
dataset.loc[dataset['Injury'] == '0', 'injured'] = 0

# dataset[dataset['FBRefID'] == '71672fa0'].tail(60)

In [36]:
# Creating target column 'injured_in_one_week' and creating cumulative features
def shift_by_time_period(df, shift_factor, column):
    df[column + '_in_' + str(shift_factor) + '_week'] = df.groupby('FBRefID')[column].shift(shift_factor*-1)
    return df

dataset = shift_by_time_period(dataset, 1, 'injured')
dataset = shift_by_time_period(dataset, 4, 'injured')
dataset = shift_by_time_period(dataset, 12, 'injured')
dataset = shift_by_time_period(dataset, 26, 'injured')
dataset = shift_by_time_period(dataset, 52, 'injured')

dataset = shift_by_time_period(dataset, 1, 'injury_count')
dataset = shift_by_time_period(dataset, 4, 'injury_count')
dataset = shift_by_time_period(dataset, 12, 'injury_count')
dataset = shift_by_time_period(dataset, 26, 'injury_count')
dataset = shift_by_time_period(dataset, 52, 'injury_count')

dataset = shift_by_time_period(dataset, 1, 'cum_injury')
dataset = shift_by_time_period(dataset, 4, 'cum_injury')
dataset = shift_by_time_period(dataset, 12, 'cum_injury')
dataset = shift_by_time_period(dataset, 26, 'cum_injury')
dataset = shift_by_time_period(dataset, 52, 'cum_injury')

# dataset['injured_in_one_week'] = dataset.groupby('FBRefID').injured.shift(-1)
# dataset[dataset['injured_in_one_week'].isna()] = 0

# dataset[dataset['FBRefID'] == 'dea698d9'].tail(60)

In [37]:
# #   reset  val  desired_col
# #0      0    1            1
# #1      0    5            6
# #2      0    4           10
# #3      1    2            2
# #4      1   -1           -1
# #5      0    6            5
# #6      0    4            9
# #7      1    2            2
# df['cumsum'] = df['reset'].cumsum()
# #cumulative sums of groups to column des
# df['des']= df.groupby(['cumsum'])['val'].cumsum()
# print df
# #   reset  val  desired_col  cumsum  des
# #0      0    1            1       0    1
# #1      0    5            6       0    6
# #2      0    4           10       0   10
# #3      1    2            2       1    2
# #4      1   -1           -1       2   -1
# #5      0    6            5       2    5
# #6      0    4            9       2    9
# #7      1    2            2       3    2
# #remove columns desired_col and cumsum
# df = df.drop(['desired_col', 'cumsum'], axis=1)
# print df
# #   reset  val  des
# #0      0    1    1
# #1      0    5    6
# #2      0    4   10
# #3      1    2    2
# #4      1   -1   -1
# #5      0    6    5
# #6      0    4    9
# #7      1    2    2

In [38]:
# Creating 'cum_sum' column to serve as base for cummulative features

dataset['cum_sum'] = dataset['injured'].cumsum()

# dataset['Min_cum'] = dataset.groupby(['FBRefID', 'cum_sum'])['Min'].cumsum()


In [39]:
# Creating function to add cummulative columns

def cummulative_sum(dataset, cum_column, original_column):
    dataset[cum_column] = dataset.groupby(['FBRefID', 'cum_sum'])[original_column].cumsum()
    return dataset

In [40]:
# Creating cummulative variables
cum_cols = ['Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Press', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xA', 
    'SCA', 'GCA', 'Cmp', 'Att', 'Prog', 'Carries', 'Prog.1', 'Succ', 'Att.1', 'Fls', 'Fld', 'Off', 'Crs', 'TklW', 'OG', 'PKwon', 'PKcon', 'Won', 
    'Loss', 'Draw', 'was_match']

for var in cum_cols:
    cummulative_sum(dataset, var+'_cum', var)

In [41]:
dataset.head()

,name,FBRefID,date,agg_week,agg_year,Injury,injury_week,injury_year,Min,Gls,...,Off_cum,Crs_cum,TklW_cum,OG_cum,PKwon_cum,PKcon_cum,Won_cum,Loss_cum,Draw_cum,was_match_cum
0,Marco Benassi,0000acda,2012-08-05,31.0,2012.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,Marco Benassi,0000acda,2012-08-12,32.0,2012.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0
2,Marco Benassi,0000acda,2012-08-19,33.0,2012.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0
3,Marco Benassi,0000acda,2012-08-26,34.0,2012.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,3.0
4,Marco Benassi,0000acda,2012-09-02,35.0,2012.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,4.0


In [42]:
# Saving dataset_for_model_final to Google Drive

# Renzo's path
# dataset.to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/dataset_for_model_final.csv', index=False)

# Sravan's path
dataset.to_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/dataset_for_model_final.csv', index=False)